In [2]:
print(os.getcwd())

C:\ai_x\source\Project


In [12]:
# 폴더 취합
import os
import shutil


src_root = r'C:\ai_x\source\Project\모델링 테스트용\raw_data'
dst_root = r'C:\ai_x\source\Project\모델링 테스트용\clean_data'

print("src_root 존재 여부 :", os.path.exists(src_root))
print("dst_root 존재 여부 :", os.path.exists(dst_root))
print("폴더 목록:", os.listdir(src_root) if os.path.exists(src_root) else "❌ 폴더 없음")
os.makedirs(dst_root, exist_ok=True)

for store in os.listdir(src_root):
    store_path = os.path.join(src_root, store)
    dst_store_path = os.path.join(dst_root, store)
    os.makedirs(dst_store_path, exist_ok=True)

    for subfolder in os.listdir(store_path):
        subfolder_path = os.path.join(store_path, subfolder)

        if os.path.isdir(subfolder_path):
            for img_file in os.listdir(subfolder_path):
                src_img = os.path.join(subfolder_path, img_file)
                dst_img = os.path.join(dst_store_path, f"{subfolder}_{img_file}")
                shutil.copy(src_img, dst_img)

src_root 존재 여부 : True
dst_root 존재 여부 : True
폴더 목록: ['KFC 신림역', '나인온스버거 서울대500동점', '롯데리아 구로디지털역점', '롯데리아 신림역점', '맥도날드 신림점', '버거락 신림역점', '버거운버거 서울대점', '버거킹 신림역점', '쉐이크쉑 신림점', '아토커피 신림점']


In [20]:
print(os.getcwd())

C:\ai_x\source\Project


In [21]:
import os
import shutil
from sklearn.model_selection import train_test_split

original_dir = r"C:\ai_x\source\Project\모델링 테스트용\clean_data" # 모델링용 테스트 이미지 데이터 

# 분할 결과를 저장할 폴더
base_dir = r"C:\ai_x\source\Project\모델링 테스트용\test_sample_data"
for split in ['train', 'val', 'test']:
    for store in os.listdir(original_dir):
        os.makedirs(os.path.join(base_dir, split, store), exist_ok=True)

# 분할 비율 설정
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

for store in os.listdir(original_dir):
    store_path = os.path.join(original_dir, store)
    images = os.listdir(store_path)

    # train / val+test 분할
    train_imgs, val_test_imgs = train_test_split(images, test_size=(1 - train_ratio), random_state=42)

    # val / test 분할
    val_imgs, test_imgs = train_test_split(val_test_imgs, test_size=test_ratio/(test_ratio + val_ratio), random_state=42)

    # 이미지 복사 함수
    def copy_images(img_list, split):
        for img in img_list:
            src = os.path.join(store_path, img)
            dst = os.path.join(base_dir, split, store, img)
            shutil.copy(src, dst)

    copy_images(train_imgs, 'train')
    copy_images(val_imgs, 'val')
    copy_images(test_imgs, 'test')

print("데이터셋 분할 완료!")

데이터셋 분할 완료!


In [24]:
print(os.getcwd())

C:\ai_x\source\Project


In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 경로
base_dir = r"C:\ai_x\source\Project\모델링 테스트용\test_sample_data"
# # 경로 확인용 출력
# print("train 폴더 존재 여부:", os.path.exists(os.path.join(base_dir, "train")))

# 전처리 + 자동 라벨링
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# 훈련 데이터
train_gen = datagen.flow_from_directory(
    base_dir + "/train",
    target_size=(224, 224),
    batch_size=16,
    class_mode="categorical",
    subset="training"
)
# 검증 데이터 
val_gen = datagen.flow_from_directory(
    base_dir + "/train",
    target_size=(224, 224),
    batch_size=16,
    class_mode="categorical",
    subset="validation"
)

Found 436 images belonging to 10 classes.
Found 105 images belonging to 10 classes.


In [27]:
# 모델 만들기 
from tensorflow.keras import layers,models,Sequential

model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [28]:
# 모델 학습
model.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10
28/28 [==============================] - 27s 924ms/step - loss: 5.4438 - accuracy: 0.2592 - val_loss: 2.0660 - val_accuracy: 0.3524
Epoch 2/10
28/28 [==============================] - 21s 761ms/step - loss: 1.3497 - accuracy: 0.5872 - val_loss: 2.2152 - val_accuracy: 0.3905
Epoch 3/10
28/28 [==============================] - 26s 931ms/step - loss: 0.8030 - accuracy: 0.7317 - val_loss: 2.1743 - val_accuracy: 0.5238
Epoch 4/10
28/28 [==============================] - 23s 832ms/step - loss: 0.4425 - accuracy: 0.8394 - val_loss: 2.6260 - val_accuracy: 0.5238
Epoch 5/10
28/28 [==============================] - 25s 898ms/step - loss: 0.3206 - accuracy: 0.9037 - val_loss: 2.5290 - val_accuracy: 0.5238
Epoch 6/10
28/28 [==============================] - 24s 851ms/step - loss: 0.3049 - accuracy: 0.9014 - val_loss: 3.4181 - val_accuracy: 0.4762
Epoch 7/10
28/28 [==============================] - 23s 813ms/step - loss: 0.1935 - accuracy: 0.9312 - val_loss: 3.2174 - val_accuracy: 0.4952

In [31]:
# 이미지 넣고 예측
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img(r"C:\ai_x\source\Project\모델링 테스트용\test_sample_data\test\버거운버거 서울대점\리뷰_이미지_개별리뷰_41_02.jpg", target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # [1, 224, 224, 3]

prediction = model.predict(img_array)
class_idx = np.argmax(prediction)

# 클래스 이름 확인
class_names = list(train_gen.class_indices.keys())
print("예측된 가게:", class_names[class_idx])


1/1 [==============================] - 0s 43ms/step
예측된 가게: 버거락 신림역점
